In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import json

In [3]:
with open('./benchmark_data/MA_journal_net.txt') as MA_journal_flow:
    MA_journal_flow_lines = MA_journal_flow.readlines()
MA_journal_links = []
i = -9
for line in MA_journal_flow_lines:
    i += 1
    if i > 0:
        MA_journal_links.append(line.split('  ')[1:3])
numLinks = i

In [4]:
MA_journal_links;

In [5]:
link_list_js = [str(int(MA_journal_links[i][0])) + ',' + str(int(MA_journal_links[i][1])) for \
             i in range(len(MA_journal_links))]

link_list_pk = [str(int(MA_journal_links[i][0])) + '->' + str(int(MA_journal_links[i][1])) for \
             i in range(len(MA_journal_links))]

In [6]:
zdump(link_list_js, '../temp_files/MA_journal_link_list_js.pkz')

zdump(link_list_pk, '../temp_files/MA_journal_link_list_pk.pkz')

In [7]:
numNodes = max([int(MA_journal_links[i][1]) for i in range(numLinks)])

In [8]:
numNodes

74

In [9]:
link_list_js;

In [10]:
from collections import defaultdict

node_neighbors_dict = defaultdict(list)

for node in range(numNodes):
    for link in MA_journal_links:
        if node == int(link[0]):
            node_neighbors_dict[str(node)].append(int(link[1]))

In [11]:
node_neighbors_dict['10']

[5, 11, 18, 20]

In [12]:
with open('./benchmark_data/MA_journal_trips.txt') as MA_journal_trips:
    MA_journal_trips_lines = MA_journal_trips.readlines()

In [13]:
numZones = int(MA_journal_trips_lines[0].split(' ')[3])

In [14]:
numZones

74

In [15]:
od_pairs = []
for i in range(numZones+1)[1:]:
    for j in range(numZones+1)[1:]:
        if i != j:
            od_pairs.append([i, j])

In [16]:
od_pairs;

In [17]:
# create O-D pair labels
# create a dictionary mapping O-D pairs to labels

import json

OD_pair_label_dict = {}
OD_pair_label_dict_ = {}

label = 1
for i in range(numZones + 1)[1:]:
    for j in range(numZones + 1)[1:]:
        key = (i, j)
        OD_pair_label_dict[str(key)] = label
        OD_pair_label_dict_[str(label)] = key
        label += 1
        
with open('../temp_files/od_pair_label_dict_MA_journal.json', 'w') as json_file:
    json.dump(OD_pair_label_dict, json_file)
    
with open('../temp_files/od_pair_label_dict__MA_journal.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_, json_file)


OD_pair_label_dict_refined = {}
OD_pair_label_dict_refined_ = {}

label = 1
for i in range(numZones + 1)[1:]:
    for j in range(numZones + 1)[1:]:
        if i != j:
            key = (i, j)
            OD_pair_label_dict_refined[str(key)] = label
            OD_pair_label_dict_refined_[str(label)] = key
            label += 1
        
with open('../temp_files/od_pair_label_dict_MA_journal_refined.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_refined, json_file)
    
with open('../temp_files/od_pair_label_dict__MA_journal_refined.json', 'w') as json_file:
    json.dump(OD_pair_label_dict_refined_, json_file)
    
    
# create link labels
# create a dictionary mapping directed links to labels
link_label_dict = {}
link_label_dict_ = {}

link_list = zload('../temp_files/MA_journal_link_list_js.pkz')

for i in range(numLinks):
    link_label_dict[str(i)] = link_list[i]

for i in range(numLinks):
    link_label_dict_[link_list[i]] = i

with open('../temp_files/link_label_dict_MA_journal.json', 'w') as json_file:
    json.dump(link_label_dict, json_file)
    
with open('../temp_files/link_label_dict_MA_journal_.json', 'w') as json_file:
    json.dump(link_label_dict_, json_file)
    
# create link labels
# create a dictionary mapping directed links to labels
link_label_dict = {}
link_label_dict_ = {}

link_list = zload('../temp_files/MA_journal_link_list_pk.pkz')

for i in range(numLinks):
    link_label_dict[str(i)] = link_list[i]

for i in range(numLinks):
    link_label_dict_[link_list[i]] = i

zdump(link_label_dict, '../temp_files/link_label_dict_MA_journal_network.pkz')
zdump(link_label_dict_, '../temp_files/link_label_dict_MA_journal_network_.pkz')

In [18]:
link_label_dict_;

In [19]:
link_length_dict_MA_journal = {}  # save free-flow time actually
link_capac_dict_MA_journal = {}

length_list = []
capac_list = []
with open('./benchmark_data/MA_journal_net.txt', 'r') as f:
    read_data = f.readlines()
    flag = 0
    for row in read_data:
        if ';' in row:
            flag += 1
            if flag > 1:
                length_list.append(float(row.split('  ')[5]))
                capac_list.append(float(row.split('  ')[3]))
                
for idx in range(len(length_list)):
    key = str(idx)
    link_length_dict_MA_journal[key] = length_list[idx]
    link_capac_dict_MA_journal[key] = capac_list[idx]

with open('../temp_files/link_length_dict_MA_journal.json', 'w') as json_file:
    json.dump(link_length_dict_MA_journal, json_file)
    
with open('../temp_files/link_capac_dict_MA_journal.json', 'w') as json_file:
    json.dump(link_capac_dict_MA_journal, json_file)

In [20]:
link_length_dict_MA_journal;

In [21]:
link_label_dict = zload('../temp_files/link_label_dict_MA_journal_network.pkz')
link_label_dict_ = zload('../temp_files/link_label_dict_MA_journal_network_.pkz')

In [22]:
keyLink = str(link_label_dict_['10->5'])
link_length_dict_MA_journal[keyLink]

0.148699

In [23]:
# compute length of a route
def routeLength(route):
    link_list = []
    node_list = []
    for i in route.split('->'):
        node_list.append(int(i))
    for i in range(len(node_list))[:-1]:
        link_list.append('%d->%d' %(node_list[i], node_list[i+1]))
    length_of_route = sum([link_length_dict_MA_journal[str(link_label_dict_[link])] for link in link_list])
    return length_of_route

In [24]:
routeLength('10->5')

0.148699

In [25]:
node_neighbors_dict;

In [26]:
len(length_list)

258

In [27]:
import networkx as nx

MA_journal = nx.DiGraph()

MA_journal.add_nodes_from(range(numNodes+1)[1:])

MA_journal_weighted_edges = [(int(link_list_js[i].split(',')[0]), int(link_list_js[i].split(',')[1]), \
                           length_list[i]) for i in range(len(link_list_js))]

MA_journal.add_weighted_edges_from(MA_journal_weighted_edges)

path = nx.all_pairs_dijkstra_path(MA_journal)

with open('../temp_files/path-link_incidence_MA_journal.txt', 'w') as the_file:
    for od in od_pairs:
        origi = od[0]
        desti = od[1]
        the_file.write('O-D pair (%s, %s):\n'%(origi, desti))
        route = str(path[origi][desti]).replace("[", "").replace(", ", "->").replace("]", "")
        the_file.write(route)
        the_file.write('\n')

In [28]:
link_list_js;

In [29]:
with open('../temp_files/path-link_incidence_MA_journal.txt', 'r') as the_file:
    # path counts
    i = 0  
    for row in the_file:
        if '->' in row:
            i = i + 1

with open('../temp_files/numRoutes_MA_journal.json', 'w') as json_file:
    json.dump(i, json_file)